# Diabetic Retinopathy Project

In [2]:
from tensorflow.keras.layers import Dense,Flatten,Input

In [3]:
from tensorflow.keras.models import Model

In [4]:
from tensorflow.keras.preprocessing import image

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [6]:
from tensorflow.keras.applications.xception import Xception,preprocess_input

In [7]:
from glob import glob

In [8]:
import numpy as np

In [9]:
import matplotlib.pyplot as plt

In [10]:
imageSize=[299,299]

In [11]:
trainPath=r"/dataset/training"

In [12]:
testPath=r"/dataset/testing"

In [13]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [14]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [15]:
training_set = train_datagen.flow_from_directory(
    'dataset/training',
    target_size=(299,299),
    batch_size=32,
    class_mode='categorical'
)

Found 3662 images belonging to 5 classes.


In [16]:
test_set = test_datagen.flow_from_directory(
    'dataset/testing',
    target_size=(299,299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 734 images belonging to 5 classes.


In [17]:
xception=Xception(input_shape=imageSize+[3],weights='imagenet',include_top=False)

In [18]:
for layer in xception.layers:
  layer.trainable=False

In [19]:
x=Flatten()(xception.output)

In [20]:
prediction=Dense(5,activation='softmax')(x)

In [21]:
model=Model(inputs=xception.input,outputs=prediction)

In [22]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 21,885,485 (83.49 MB)

 Trainable params: 1,024,005 (3.91 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30
)


Epoch 1/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1220s 11s/step - accuracy: 0.6546 - loss: 4.7078 - val_accuracy: 0.7439 - val_loss: 2.7104
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1243s 11s/step - accuracy: 0.7196 - loss: 3.0471 - val_accuracy: 0.7262 - val_loss: 2.2101
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1196s 10s/step - accuracy: 0.7490 - loss: 2.7694 - val_accuracy: 0.7888 - val_loss: 1.6767
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1161s 10s/step - accuracy: 0.7433 - loss: 3.2813 - val_accuracy: 0.7480 - val_loss: 3.4207
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1125s 10s/step - accuracy: 0.7695 - loss: 2.9027 - val_accuracy: 0.7698 - val_loss: 3.4961
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1088s 9s/step - accuracy: 0.7769 - loss: 3.0604 - val_accuracy: 0.7738 - val_loss: 3.5395
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1052s 9s/step - accuracy: 0.7671 - loss: 3.4780 - val_accuracy: 0.7861 - val_loss: 2.6202
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 1211s 11s/step - accuracy: 0.7971 - loss: 2.8

In [25]:
model.save('Updated-Xception-diabetic-retinopathy.h5')

In [1]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299, 3)
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [2]:
model.save("Updated-xception-diabetic-retinopathy.h5")